In [1]:
import garpar as gp
import itertools as it
import pandas as pd
import numpy as np
import joblib as jb
import tqdm.notebook
import scipy 
import matplotlib.pyplot as plt
import seaborn as sns
from lib import lsboostrap

In [2]:
merval21 = gp.datasets.load_MERVAL(
    first="01-02-2021", last = "12-31-2021")
merval21 = merval21.as_returns()
merval21

Stocks,ALUA,BBAR,BYMA,CEPU,COME,CRES,CVH,EDN,GGAL,HARG,...,MOLA,MOLI,MORI,OEST,PATA,POLL,RIGO,SAMI,SEMI,TGLT
Days,,,,,,,,,,,,,,,,,,,,,
2021-01-05,0.056995,-0.014122,-0.003331,0.039242,0.004132,0.027451,0.020047,-0.012456,0.003245,0.016598,...,0.028148,0.006873,0.046358,-0.001789,0.021505,0.076503,0.000000,0.000000,-0.010889,-0.006061
2021-01-06,0.035294,-0.012325,0.008354,-0.026042,0.008230,0.013740,0.035838,-0.007207,-0.004852,0.016327,...,0.011527,-0.010239,-0.006329,-0.017921,-0.026316,0.000000,0.015385,0.008475,0.023853,0.006098
2021-01-07,0.005682,-0.015514,-0.000829,-0.005348,-0.004082,0.038404,0.001116,-0.014519,0.006908,-0.006024,...,0.001425,-0.006897,0.054140,0.000000,0.000000,0.000000,0.030303,0.000000,0.109319,-0.012121
2021-01-08,0.005650,-0.002398,-0.000829,-0.013441,-0.012295,-0.015228,-0.125975,-0.009208,-0.010089,-0.004040,...,-0.001422,0.005787,-0.012085,-0.023723,-0.005405,0.000000,-0.011029,0.008403,-0.046850,-0.053170
2021-01-11,0.007491,-0.050137,-0.004149,-0.016349,-0.004149,0.002946,-0.019133,-0.040892,-0.019568,-0.012170,...,-0.027066,-0.016110,-0.024465,-0.009346,0.000000,-0.005076,-0.011152,0.008333,0.033898,0.010799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,0.024202,0.020322,0.031205,0.029366,0.001681,0.000996,0.021193,0.024510,0.003778,0.004280,...,0.016570,0.012295,-0.024922,0.048493,0.056319,0.000000,-0.006579,-0.001786,0.005025,0.008021
2021-12-27,-0.008593,0.019689,-0.017691,-0.010511,0.028523,-0.056716,-0.039969,0.029665,0.004266,-0.036932,...,0.029956,0.000000,0.012780,0.018750,-0.016905,-0.024299,0.000000,0.016100,0.000000,-0.013263
2021-12-28,-0.050921,-0.049394,-0.032227,-0.042489,0.001631,0.007911,-0.013611,-0.036245,-0.034733,-0.022124,...,0.028229,0.004049,0.000000,-0.019632,-0.010582,-0.007663,0.000000,-0.019366,0.001667,0.008065


In [3]:
# definimos los sectores 
sectores = {'come': ["AUSO", "BOLT", "DYCA","GCLA", "IRCP", "IRSA", "OEST", "PATA", "POLL","TECO2","CVH"], 
'moa': ["CELU", "GRIM", "HAVA", "LEDE", "MOLA", "MOLI", "MORI", "SAMI", "SEMI"],
'moi': ["AGRO", "ALUA",  "FERR", "LONG", "RIGO", "HARG", "FIPL"],
'agro': ["CADO", "GARO", "INVJ","CRES"], 
'ene': ["CEPU", "CGPA2", "DGCU2", "GBAN", "METR", "TGNO4", "TGSU2", "YPFD", "PAMP","TRAN","COME","EDN","GBAN" ], 
'fin': ["BHIP", "BPAT",  "BYMA", "GGAL", "SUPV", "BBAR"] }

In [5]:
for sector,stocks in sectores:
    stocks = stocks
    result= lsboostrap.levy_stable(
    df=merval21, frac=0.8, columns=stocks, iterations=2, rstate=1, n_jobs=1)
    filename = f"2021_{sector}.csv"
    jb.dump(result,filename)

ValueError: too many values to unpack (expected 2)

In [ ]:
come = lsboostrap.levy_stable(
    df=merval21, frac=0.8, columns=come, iterations=2, rstate=1, n_jobs=1)